In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, Bidirectional, GRU, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import polars as pl

def load_and_preprocess_data(train_file, dev_file, test_file, max_features, max_words):
    # Load the data
    data_train = pl.read_csv(train_file)
    data_dev = pl.read_csv(dev_file)
    data_test = pl.read_csv(test_file)

    # Ensure the Tweet column is in string format
    data_train = data_train.with_column(pl.col("Tweet").cast(pl.Utf8))
    data_dev = data_dev.with_column(pl.col("Tweet").cast(pl.Utf8))
    data_test = data_test.with_column(pl.col("Tweet").cast(pl.Utf8))

    # Convert target variables to categorical
    y_train = to_categorical(data_train['Intensity_Class'].to_numpy())
    y_val = to_categorical(data_dev['Intensity_Class'].to_numpy())
    y_test = to_categorical(data_test['Intensity_Class'].to_numpy())

    # Prepare the training, validation, and test data
    X_train = data_train['Tweet'].to_list()
    X_val = data_dev['Tweet'].to_list()
    X_test = data_test['Tweet'].to_list()

    # Tokenize the text data
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_val_seq = tokenizer.texts_to_sequences(X_val)
    X_test_seq = tokenizer.texts_to_sequences(X_test)

    # Pad sequences
    X_train_pad = pad_sequences(X_train_seq, maxlen=max_words)
    X_val_pad = pad_sequences(X_val_seq, maxlen=max_words)
    X_test_pad = pad_sequences(X_test_seq, maxlen=max_words)

    return X_train_pad, y_train, X_val_pad, y_val, X_test_pad, y_test

def build_and_compile_model(max_features, max_words, num_classes):
    # Define the Bidirectional GRU model
    model = Sequential()
    model.add(Embedding(max_features, 100, input_length=max_words))
    model.add(SpatialDropout1D(0.25))
    model.add(Bidirectional(GRU(64, dropout=0.4, return_sequences=True)))
    model.add(Bidirectional(GRU(32, dropout=0.5, return_sequences=False)))
    model.add(Dense(num_classes, activation='sigmoid'))

    # Compile the model with a smaller learning rate
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

    return model
